In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci
import CC, Wick
from math import factorial, sqrt

In [2]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [3]:
mf.energy_tot()

-1.1026388111917238

In [4]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [5]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [6]:
eri

array([[6.54009511e-01, 8.32667268e-17, 6.45249427e-01],
       [1.04083409e-16, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 1.38777878e-16, 6.78136184e-01]])

In [7]:
Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1 for i in range(Nocc)] + [0 for i in range (Norbs - Nocc)]

In [8]:
integralTensor2body = np.zeros((Norbs, Norbs, Norbs, Norbs))
for p in range(Norbs):
    for q in range(Norbs):
        for r in range(p + 1):
            for s in range(q + 1):
                x = int(p + Norbs * r - 0.5 * r * (r + 1))
                y = int(q + Norbs * s - 0.5 * s * (s + 1))
                print(p, q, r, s)
                print(x, y)
                if p == r and q == s:
                    integralTensor2body[p,q,r,s] = eri[x, y]
#                    hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s)
                else:
                    integralTensor2body[p,q,r,s] = eri[x, y]
                    integralTensor2body[r,s,p,q] = np.conjugate(eri[x, y])
#                    hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s) + 0.5 * np.conjugate(eri[x, y]) * spinFreeDoubleExcitation(r, s, p, q)

0 0 0 0
0 0
0 1 0 0
0 1
0 1 0 1
0 2
1 0 0 0
1 0
1 0 1 0
2 0
1 1 0 0
1 1
1 1 0 1
1 2
1 1 1 0
2 1
1 1 1 1
2 2


In [9]:
integralTensor2body

array([[[[6.54009511e-01, 8.32667268e-17],
         [1.04083409e-16, 1.87521981e-01]],

        [[8.32667268e-17, 6.45249427e-01],
         [0.00000000e+00, 5.55111512e-17]]],


       [[[1.04083409e-16, 0.00000000e+00],
         [6.45249427e-01, 1.38777878e-16]],

        [[1.87521981e-01, 5.55111512e-17],
         [1.38777878e-16, 6.78136184e-01]]]])

In [10]:
integralTensor2body[1,0,1,0]

0.6452494265884844

In [11]:
UGgenerators1body = np.ndarray((Norbs, Norbs), dtype=Wick.operatorSum)
for p in range(Norbs):
    for q in range(Norbs):
        UGgenerators1body[p,q] = CC.spinFreeNtupleExcitation((p,), (q,))

In [12]:
UGgenerators2body = np.ndarray((Norbs, Norbs, Norbs, Norbs), dtype=Wick.operatorSum)
for p in range(Norbs):
    for q in range(Norbs):
        for r in range(Norbs):
            for s in range(Norbs):
                UGgenerators2body[p,q,r,s] = CC.spinFreeNtupleExcitation((p, q), (r, s))

In [13]:
print(UGgenerators2body[0,1,1,0])

1.0 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{1\alpha}
 + 1.0 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 1.0 * a^{0\beta}a^{1\alpha}a_{0\alpha}a_{1\beta}
 + 1.0 * a^{0\beta}a^{1\beta}a_{0\beta}a_{1\beta}


In [38]:
class Index:
    '''
    Class for an orbital index
    Orbital types are 'g' (general), 'h' (inactive hole), 'a' (active), and 'p' (particle)
    '''
    def __init__(self, name, orbitalType):
        self.name = name
        self.orbitalType = orbitalType

    def __str__(self):
        return self.name
#        return self.orbitalType + str(self.indexNumber)


In [238]:
class Tensor:
    '''
    Class for amplitude tensors of spin-free excitation operators
    '''
    def __init__(self, name, lowerIndicesTuple, upperIndicesTuple, range):
        self.name = name
        self.lowerIndices = lowerIndicesTuple
        self.upperIndices = upperIndicesTuple
        self.excitationRank = len(self.lowerIndices)
        self.range = range
        self.array = np.zeros(tuple([self.range] * (2 * self.excitationRank)))

    def setArray(self, array):
        if array.shape == self.array.shape:
            self.array = array
        else:
            print("Array is of wrong shape")

    def getOperator(self):
        operator = Wick.operatorSum([])
        it = np.nditer(self.array, flags=["multi_index"])
        
        for i in it:
            indices = it.multi_index
            creationIndices = indices[:self.excitationRank]
            annihilationIndices = indices[self.excitationRank:]
            operator += i * CC.spinFreeNtupleExcitation(creationIndices, annihilationIndices)
        return operator

    def __str__(self):
        string = self.name + "_{"
        for p in self.lowerIndices:
            string += p.__str__()
        string += "}^{"
        for q in self.upperIndices:
            string += q.__str__()
        string += "}"
        return string

In [178]:
class productOfTensors:
    def __init__(self, listOfTensors):
        self.name = name
        self.listOfTensors = listOfTensors

    def __str__(self):
        string = ""
        for t in self.listOfTensors:
            string += t.__str__()
        return string

In [156]:
p = Index('p','g')
q = Index('q','g')
r = Index('r','g')
s = Index('s','g')

In [157]:
hpq = Tensor('h', (p,), (q,), Norbs)

In [158]:
print(hpq)

h_{p}^{q}


In [159]:
hpq.setArray(h1)

In [160]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [161]:
print(hpq.getOperator())

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}


In [162]:
gpqrs = Tensor('g', (p, q), (r,s), Norbs)
print(gpqrs)

g_{pq}^{rs}


In [163]:
print(h.array)

[[0. 0.]
 [0. 0.]]


In [164]:
i = Index('i', 'g')
j = Index('j', 'g')
Aij = Tensor('A',(i,),(j,),2)
print(Aij.array)
Aij.setArray(np.arange(4).reshape(2,2))
print(Aij.array)

[[0. 0.]
 [0. 0.]]
[[0 1]
 [2 3]]


In [177]:
k = Index('k', 'g')
l = Index('l', 'g')
Bkl = Tensor('B',(k,),(l,),2)
print(Bkl.array)
Bkl.setArray(np.arange(4).reshape(2,2))
print(Bkl.array)

[[0.0e+000 4.9e-324]
 [9.9e-324 1.5e-323]]
[[0 1]
 [2 3]]


In [165]:
Aij.lowerIndices

(<__main__.Index at 0x10dafce50>,)

In [166]:
j != Aij.lowerIndices[0]

True

In [167]:
def getAxis(tensor, index):
    for a in range(tensor.excitationRank):
        if tensor.lowerIndices[a] == index:
            return a
        elif tensor.upperIndices[a] == index:
            return tensor.excitationRank + a

In [176]:
getAxis(gpqrs, s)

3

In [171]:
gpqrs = Tensor('g',(p,q), (r,s), Norbs)
print(gpqrs.array)
gpqrs.setArray(integralTensor2body)
print(gpqrs.array)

[[[[6.54009511e-01 8.32667268e-17]
   [1.04083409e-16 1.87521981e-01]]

  [[8.32667268e-17 6.45249427e-01]
   [0.00000000e+00 5.55111512e-17]]]


 [[[1.04083409e-16 0.00000000e+00]
   [6.45249427e-01 1.38777878e-16]]

  [[1.87521981e-01 5.55111512e-17]
   [1.38777878e-16 6.78136184e-01]]]]
[[[[6.54009511e-01 8.32667268e-17]
   [1.04083409e-16 1.87521981e-01]]

  [[8.32667268e-17 6.45249427e-01]
   [0.00000000e+00 5.55111512e-17]]]


 [[[1.04083409e-16 0.00000000e+00]
   [6.45249427e-01 1.38777878e-16]]

  [[1.87521981e-01 5.55111512e-17]
   [1.38777878e-16 6.78136184e-01]]]]


In [131]:
Wick.vacuumExpectationValue(hpq.getOperator() + 0.5 * gpqrs.getOperator(), [1,0]) + Enuc

-1.1026388111917234

In [201]:
def contract(Tensor1, Tensor2, index1, index2):
    axis1 = getAxis(Tensor1, index1)
    axis2 = getAxis(Tensor2, index2)
    newLowerIndicesList = list(Tensor1.lowerIndices)
    newLowerIndicesList.remove(index1)
    newLowerIndicesList = newLowerIndicesList + list(Tensor2.lowerIndices)
    newUpperIndicesList = list(Tensor2.upperIndices)
    newUpperIndicesList.remove(index2)
    newUpperIndicesList = list(Tensor1.upperIndices) + newUpperIndicesList
    name = Tensor1.name + Tensor2.name
    resultArray = np.tensordot(Tensor1.array, Tensor2.array, axes=(axis1, axis2))
    result = Tensor(name, newLowerIndicesList, newUpperIndicesList, Norbs)
    result.setArray(resultArray)
    return result

In [209]:
vacuum = np.array([1,0])

In [214]:
Wick.vacuumExpectationValue(gpqrs.getOperator(), vacuum)

1.3080190227122477

In [213]:
2 * np.tensordot(np.tensordot(vacuum, hpq.array, axes=1), vacuum, axes=1)

-2.379701237214037

In [216]:
2 * np.tensordot(np.tensordot(np.tensordot(np.tensordot(vacuum, gpqrs.array, axes=1), vacuum, axes=1), vacuum, axes=1), vacuum, axes=1)

1.3080190227122477

In [197]:
print(Aij)
print(Aij.array)
print(Bkl)
print(Bkl.array)

A_{i}^{j}
[[0 1]
 [2 3]]
B_{k}^{l}
[[0 1]
 [2 3]]


In [208]:
print(contract(Aij, Bkl, i, l))
print(contract(Aij, Bkl, i, l).getOperator())

AB_{k}^{j}
2.0 * a^{0\alpha}a_{0\alpha}
 + 6.0 * a^{0\alpha}a_{1\alpha}
 + 2.0 * a^{0\beta}a_{0\beta}
 + 6.0 * a^{0\beta}a_{1\beta}
 + 3.0 * a^{1\alpha}a_{0\alpha}
 + 11.0 * a^{1\alpha}a_{1\alpha}
 + 3.0 * a^{1\beta}a_{0\beta}
 + 11.0 * a^{1\beta}a_{1\beta}


In [207]:
print(gpqrs.array)

[[[[6.54009511e-01 8.32667268e-17]
   [1.04083409e-16 1.87521981e-01]]

  [[8.32667268e-17 6.45249427e-01]
   [0.00000000e+00 5.55111512e-17]]]


 [[[1.04083409e-16 0.00000000e+00]
   [6.45249427e-01 1.38777878e-16]]

  [[1.87521981e-01 5.55111512e-17]
   [1.38777878e-16 6.78136184e-01]]]]


In [180]:
t = Tensor(2, 2, 3)

TypeError: __init__() missing 1 required positional argument: 'range'

In [18]:
t.array

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]])

In [19]:
t.shape

AttributeError: 'Tensor' object has no attribute 'shape'

In [12]:
u = Tensor(1, 1, 3)

In [13]:
u.array

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [20]:
np.einsum('ii', u.array)

0.0

In [21]:
a = np.arange(9).reshape((3,3))
print(a)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [22]:
b = np.arange(9).reshape((3,3))
print(b)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [39]:
np.einsum("ij,kl->il", a, b)

array([[ 27,  36,  45],
       [108, 144, 180],
       [189, 252, 315]])

In [228]:
def addNewIndex(indicesList, orbitalType):
    newIndexName = orbitalType + "_{" + str(len(indicesList)) + "}"
    indicesList.append(Index(newIndexName, orbitalType))
    return indicesList[-1]

In [235]:
generalIndices = []
particleIndices = []
holeIndices = []
activeIndices = []

In [236]:
h = Tensor('h', (addNewIndex(generalIndices, 'g'),), (addNewIndex(generalIndices, 'g'),), Norbs)
h.setArray(h1)
print(h)

h_{g_{0}}^{g_{1}}


In [237]:
g = Tensor('g', (generalIndices[0], addNewIndex(generalIndices, 'g')), (generalIndices[1], addNewIndex(generalIndices, 'g')), Norbs)
g.setArray(integralTensor2body)
print(g)

g_{g_{0}g_{2}}^{g_{1}g_{3}}


In [240]:
t2 = Tensor('t2', (addNewIndex(particleIndices, 'p'), addNewIndex(particleIndices, 'p')), (addNewIndex(holeIndices, 'h'), addNewIndex(holeIndices, 'h')), Norbs)
print(t2)

t2_{p_{2}p_{3}}^{h_{2}h_{3}}


In [245]:
t2amplitudes = np.arange(16).reshape((2,2,2,2))

In [246]:
t2new = Tensor('t2', (addNewIndex(particleIndices, 'p'), addNewIndex(particleIndices, 'p')), (addNewIndex(holeIndices, 'h'), addNewIndex(holeIndices, 'h')), Norbs)
print(t2new)

t2_{p_{4}p_{5}}^{h_{4}h_{5}}


In [249]:
t2.setArray(t2amplitudes)
t2new.setArray(t2amplitudes)

In [252]:
t2amplitudes[0,0,0,0] = 1

In [254]:
print(t2.array)

[[[[ 1  1]
   [ 2  3]]

  [[ 4  5]
   [ 6  7]]]


 [[[ 8  9]
   [10 11]]

  [[12 13]
   [14 15]]]]
